##### Copyright 2020 Google Inc.

Licensed under the Apache License, Version 2.0 (the "License").
<!--
    Licensed to the Apache Software Foundation (ASF) under one
    or more contributor license agreements.  See the NOTICE file
    distributed with this work for additional information
    regarding copyright ownership.  The ASF licenses this file
    to you under the Apache License, Version 2.0 (the
    "License"); you may not use this file except in compliance
    with the License.  You may obtain a copy of the License at

      http://www.apache.org/licenses/LICENSE-2.0

    Unless required by applicable law or agreed to in writing,
    software distributed under the License is distributed on an
    "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY
    KIND, either express or implied.  See the License for the
    specific language governing permissions and limitations
    under the License.
-->


# TFDV and Apache Beam sandbox

In [ ]:
%pip install tfx

In [ ]:
import logging
import re

import pyarrow as pa

from typing import List, Optional, Text, Union, Dict, Iterable

import apache_beam as beam
from apache_beam.runners.interactive.interactive_runner import InteractiveRunner
import apache_beam.runners.interactive.interactive_beam as ib

from apache_beam.options import pipeline_options
from apache_beam.options.pipeline_options import GoogleCloudOptions
from apache_beam.runners import DataflowRunner

from apache_beam.options.pipeline_options import PipelineOptions
from apache_beam.options.pipeline_options import SetupOptions

from apache_beam.io.gcp.internal.clients import bigquery

#from tensorflow_data_validation import types

from jinja2 import Template

import google.auth

In [ ]:
%%bigquery
select * from `covertype_dataset.covertype`
limit 100

In [ ]:
class DataDriftOptions(PipelineOptions):
    @classmethod
    def _add_argparse_args(cls, parser):
        parser.add_argument(
        '--baseline_data_location',
        help="Input for the pipeline",
        default='gs://workshop-datasets/covertype/small')
        
        parser.add_argument(
        '--target_data_table',
        help="A target BQ table with data to analyze",
        default="covertype_dataset.covertype")
        
   

In [ ]:
def generate_sampling_query(source_table_name, num_lots, lots):
  """Prepares the data sampling query."""

  sampling_query_template = """
       SELECT *
       FROM 
           `{{ source_table }}` AS cover
       WHERE 
       MOD(ABS(FARM_FINGERPRINT(TO_JSON_STRING(cover))), {{ num_lots }}) IN ({{ lots }})
       """
  query = Template(sampling_query_template).render(
      source_table=source_table_name, num_lots=num_lots, lots=str(lots)[1:-1])

  return query


In [ ]:
source_table_name = 'mlops-dev-env.covertype_dataset.covertype'
num_lots = 100
lots = [1]

query = generate_sampling_query(source_table_name, num_lots, lots)

In [ ]:
options = PipelineOptions()
options.view_as(GoogleCloudOptions).project = 'mlops-dev-env'

In [ ]:
custom_options = options.view_as(DataDriftOptions)

In [ ]:
#p = beam.Pipeline(options=options)

In [ ]:
p = beam.Pipeline(InteractiveRunner(), options=options)

In [ ]:
class BatchedDictsToRecordBatch(beam.DoFn):
    """DoFn to convert a batch of dictionaries to a pyarrow.RecordBatch.
  
    In the current implementation, the function relies on automatic
    coercion to pyarrow data types. 
    To be verified whether this is a right approach.
    """

    def process(self, batch: List[Dict]) -> Iterable[pa.RecordBatch]:
        
        column_names = batch[0].keys()
        values_by_column = {column_name: [] for column_name in column_names}
        for row in batch:
            for key, value in row.items():
                values_by_column[key].append(value)
                
        arrow_arrays = [
            pa.array(arr) for arr in values_by_column.values()
            ]  
        
        yield pa.RecordBatch.from_arrays(arrow_arrays, list(column_names))
  

In [ ]:
#@beam.typehints.with_input_types(types.BeamCSVRecord)
#@beam.typehints.with_output_types(pa.RecordBatch)
class DecodeBigQuery(beam.PTransform):
    """Decodes BigQuery records into Arrow RecordBatches."""
    def __init__(
        self,
        column_names: List):
    
        if not isinstance(column_names, list):
            raise TypeError('column_names is of type %s, should be a list' %
                            type(column_names).__name__)
        self._column_names = column_names
    
    def expand(self, pcoll):
        record_batches = (
            pcoll
            | beam.Map(lambda row: {field: row[field]
                                    for field in self._column_names})
            | beam.BatchElements(min_batch_size=2, max_batch_size=3)
            | beam.ParDo(
                BatchedDictsToRecordBatch())
            )

        return record_batches

In [ ]:
records = (p
           | 'query' >> beam.io.Read(beam.io.BigQuerySource(query=query, use_standard_sql=True))
           | 'batch' >> DecodeBigQuery(['Elevation', 'Aspect'])
          )

In [ ]:
ib.show_graph(p)

In [ ]:
ib.show(records)

In [ ]:
words = (p 
         | 'read' >> beam.io.ReadFromText(custom_options.data_location)
         | 'extract' >> beam.FlatMap(lambda line: re.findall(r'[\w\']+', line.strip(), re.UNICODE))
)


In [ ]:
ib.show_graph(p)

In [ ]:
counts = (words 
          | 'count' >> beam.combiners.Count.PerElement())

In [ ]:
ib.show_graph(counts)

In [ ]:
d1 = {'a': 2, 'b': 'hello'}

In [ ]:
list(d1.keys())

In [ ]:
d1.values()

In [ ]:
from google.cloud import bigquery

client = bigquery.Client()

query_job = client.query('SELECT * FROM ({}) LIMIT 0'.format(query))
results = query_job.result()

In [ ]:
schema = {field.name: field.field_type for field in results.schema}
schema

In [ ]:
schema.values()

In [ ]:
'BOOL' in schema.values()

In [ ]:
supported_types = set(['INTEGER', 'STRING', 'BOOL'])
supported_types

In [ ]:
query_schema = set(['STRING', 'BOOL', 'STRING', 'INTEGER', 'FLOAT'])
query_schema

In [ ]:
query_schema in supported_types

In [ ]:
query_schema.issubset(supported_types)

In [ ]:
schema

In [ ]:
list(schema.keys())

In [ ]:
la = ['aa', 'bb', 'cc']
lb = ['cc', 'bb', 'aa']

In [ ]:
set(la) == set(lb)

In [ ]:
set(schema.keys())

In [1]:
import tensorflow_data_validation as tfdv

In [2]:
!ls /home/jupyter/artifact_store

beam-temp-stats-045f165c898c11ea99bb42010a8a0007
beam-temp-stats-167ef80e898f11eabd0242010a8a0007
beam-temp-stats-3215cc4e898c11ea81f342010a8a0007
beam-temp-stats-3235369a898e11ea970342010a8a0007
beam-temp-stats-775b281c899111eaa3e242010a8a0007
beam-temp-stats-7ac1a246898d11ea977542010a8a0007
beam-temp-stats-8f3ee18e898d11eaa4d242010a8a0007
beam-temp-stats-8fd865ba898e11eabdda42010a8a0007
beam-temp-stats-a8870440898e11eab60d42010a8a0007
beam-temp-stats-b3ac7060898b11ea845942010a8a0007
beam-temp-stats-c06c525c898b11ea818942010a8a0007
beam-temp-stats-d08606c6898e11eabe6942010a8a0007
beam-temp-stats-d9459202898b11ea986742010a8a0007
beam-temp-stats-e664804a898d11eabf2742010a8a0007
beam-temp-stats-eba21db6898c11ea924342010a8a0007
beam-temp-stats-fff5dc56898e11ea9e6042010a8a0007
stats


In [3]:
path = '/home/jupyter/artifact_store/stats'

In [4]:
stats = tfdv.load_statistics(path)

Instructions for updating:
Use eager execution and: 
`tf.data.TFRecordDataset(path)`


In [5]:
tfdv.visualize_statistics(stats)

In [6]:
tfdv.__version__

'0.21.5'